# Student Dropout and Academic Success Prediction

## Project Overview

This project aims to predict student dropout and academic success using machine learning techniques. The problem is formulated as a **3-class classification task**:
- **Dropout**: Students who left the institution
- **Enrolled**: Students still pursuing their degree
- **Graduate**: Students who successfully completed their degree

**Dataset Source**: [UCI ML Repository - Predict Students' Dropout and Academic Success](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success)

**Key Characteristics**:
- Information available at enrollment time (demographics, academic path, socio-economic factors)
- Academic performance at end of 1st and 2nd semesters
- Strong class imbalance
- No missing values (preprocessed dataset)

**Project Goals**:
1. Build accurate classification models to identify at-risk students early
2. Understand key factors contributing to dropout
3. Ensure fairness across protected demographic attributes
4. Provide actionable insights for intervention strategies

## Table of Contents

1. [Setup and Configuration](#1.-Setup-and-Configuration)
2. [Data Loading and Initial Inspection](#2.-Data-Loading-and-Initial-Inspection)
3. [Exploratory Data Analysis (EDA)](#3.-Exploratory-Data-Analysis)
4. [Feature Engineering](#4.-Feature-Engineering)
5. [Data Preparation](#5.-Data-Preparation)
6. [Baseline Model](#6.-Baseline-Model)
7. [Model Development](#7.-Model-Development)
8. [Model Evaluation](#8.-Model-Evaluation)
9. [Model Explainability](#9.-Model-Explainability)
10. [Fairness Analysis](#10.-Fairness-Analysis)
11. [Final Model Selection](#11.-Final-Model-Selection)
12. [Conclusions and Recommendations](#12.-Conclusions-and-Recommendations)

## 1. Setup and Configuration

### 1.1 Import Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Machine Learning - Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# Machine Learning - Evaluation
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    ConfusionMatrixDisplay
)

# Imbalanced Learning
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Explainability
import shap
from lime import lime_tabular

# Fairness
from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    demographic_parity_difference,
    equalized_odds_difference
)

# Utilities
import warnings
import os
from pathlib import Path
import pickle
from datetime import datetime

# Configuration
warnings.filterwarnings('ignore')
%matplotlib inline

### 1.2 Configuration and Settings

In [ ]:
# Random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Data configuration
DATA_PATH = "data.csv"
DELIMITER = ";"
TEST_SIZE = 0.2

MODELS_DIR = Path('models')

# Create directories if they don't exist
MODELS_DIR.mkdir(exist_ok=True)

# Plot styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.3f}'.format)

print(f"  - Random State: {RANDOM_STATE}")
print(f"  - Test Size: {TEST_SIZE}")
print(f"  - Models Directory: {MODELS_DIR}")

### 1.3 Utility Functions

In [ ]:

def save_model(model, filename):
    """
    Save trained model to the models directory.
    
    Parameters:
    -----------
    model : sklearn estimator
        Trained model to save
    filename : str
        Name of the file (without path)
    """
    filepath = MODELS_DIR / filename
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)


def load_model(filename):
    """
    Load trained model from the models directory.
    
    Parameters:
    -----------
    filename : str
        Name of the file (without path)
    
    Returns:
    --------
    model : sklearn estimator
        Loaded model
    """
    filepath = MODELS_DIR / filename
    with open(filepath, 'rb') as f:
        model = pickle.load(f)
    return model


def print_section_header(title):
    """
    Print a formatted section header.
    
    Parameters:
    -----------
    title : str
        Section title
    """
    print("\n" + "="*80)
    print(f"  {title}")
    print("="*80 + "\n")


def evaluate_model(model, X_test, y_test, model_name="Model"):
    """
    Evaluate a trained model and print comprehensive metrics.
    
    Parameters:
    -----------
    model : sklearn estimator
        Trained model
    X_test : array-like
        Test features
    y_test : array-like
        Test labels
    model_name : str
        Name of the model for display
    
    Returns:
    --------
    dict : Dictionary containing all metrics
    """
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)
    
    print_section_header(f"{model_name} - Evaluation Results")
    
    # Overall metrics
    print("Overall Metrics:")
    print(f"  Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"  Macro F1: {f1_score(y_test, y_pred, average='macro'):.4f}")
    print(f"  Weighted F1: {f1_score(y_test, y_pred, average='weighted'):.4f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Confusion matrix
    print("Confusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # Get classification report as dictionary
    report_dict = classification_report(y_test, y_pred, output_dict=True)
    
    # Return metrics dictionary
    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'macro_f1': f1_score(y_test, y_pred, average='macro'),
        'weighted_f1': f1_score(y_test, y_pred, average='weighted'),
        'classification_report': report_dict,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }



---
## Ready to Begin Analysis

The notebook is now configured and ready for data loading and exploratory analysis.

## 2. Data Loading and Initial Inspection

### 2.1 Load Dataset

In [ ]:
# Load the dataset
print_section_header("Loading Dataset")

df = pd.read_csv(DATA_PATH, delimiter=DELIMITER)

print(f"  - Shape: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"  - Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

### 2.2 Initial Data Inspection

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Dataset information
print("Dataset Info:")
print("\nColumn Names and Data Types:")
df.info()

In [ ]:
# Check for missing values
print_section_header("Missing Values Check")

missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percent
})

missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values('Missing Values', ascending=False)

if len(missing_df) == 0:
    print("No missing values found.")
else:
    print(missing_df)

### 2.3 Summary Statistics

In [ ]:
# Statistical summary of numerical features
print("Statistical Summary:")
df.describe().T

### 2.4 Target Variable Distribution

In [ ]:
# Analyze target variable distribution
print_section_header("Target Variable Distribution")

target_counts = df['Target'].value_counts()
target_percent = df['Target'].value_counts(normalize=True) * 100

target_summary = pd.DataFrame({
    'Count': target_counts,
    'Percentage': target_percent
})

print(target_summary)
print(f"\nClass Imbalance Ratio:")
print(f"  Max/Min: {target_counts.max() / target_counts.min():.2f}:1")

## 3. Exploratory Data Analysis (EDA)

### 3.1 Target Variable Visualization

In [ ]:
# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
target_counts = df['Target'].value_counts()
axes[0].bar(target_counts.index, target_counts.values, color=['#e74c3c', '#3498db', '#2ecc71'])
axes[0].set_xlabel('Target Class')
axes[0].set_ylabel('Count')
axes[0].set_title('Target Variable Distribution (Count)')
axes[0].grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, (label, value) in enumerate(target_counts.items()):
    axes[0].text(i, value + 50, str(value), ha='center', va='bottom', fontweight='bold')

# Pie chart
colors = ['#e74c3c', '#3498db', '#2ecc71']
axes[1].pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90)
axes[1].set_title('Target Variable Distribution (Percentage)')

plt.tight_layout()
plt.show()

### 3.2 Feature Type Identification

In [ ]:
# Identify feature types
print_section_header("Feature Type Identification")

# Separate features from target
X = df.drop('Target', axis=1)
y = df['Target']

# Identify numerical and categorical features
numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"Total Features: {len(X.columns)}")
print(f"  - Numerical Features: {len(numerical_features)}")
print(f"  - Categorical Features: {len(categorical_features)}")

print(f"\nNumerical Features ({len(numerical_features)}):")
print(numerical_features)

if categorical_features:
    print(f"\nCategorical Features ({len(categorical_features)}):")
    print(categorical_features)

### 3.3 Protected Attributes Identification

In [ ]:
# Identify protected/sensitive attributes for fairness analysis
print_section_header("Protected Attributes for Fairness Analysis")

protected_attributes = {
    'Gender': 'Gender',
    'Age at enrollment': 'Age',
    'Nacionality': 'Nationality',
    'International': 'International Student Status',
    'Displaced': 'Displaced Status'
}

print("Protected attributes identified for fairness analysis:")
for col, description in protected_attributes.items():
    if col in df.columns:
        unique_vals = df[col].nunique()
    else:
        print(f"  [X] {col} not found in dataset")

# Store protected attribute columns for later use
protected_cols = [col for col in protected_attributes.keys() if col in df.columns]
print(f"\nTotal protected attributes: {len(protected_cols)}")

### 3.4 Numerical Features Distribution

In [ ]:
# Visualize distributions of key numerical features
print_section_header("Numerical Features Analysis")

# Select key features for visualization
key_features = [
    'Age at enrollment',
    'Admission grade',
    'Previous qualification (grade)',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)',
    'Unemployment rate',
    'Inflation rate',
    'GDP'
]

# Filter features that exist in the dataset
key_features = [f for f in key_features if f in df.columns]

# Create subplots
n_features = len(key_features)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for idx, feature in enumerate(key_features):
    axes[idx].hist(df[feature].dropna(), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'Distribution of {feature}')
    axes[idx].grid(axis='y', alpha=0.3)

# Hide unused subplots
for idx in range(len(key_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 3.5 Academic Performance by Target Class

In [ ]:
# Analyze academic performance features by target class
print_section_header("Academic Performance by Target Class")

academic_features = [
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)'
]

academic_features = [f for f in academic_features if f in df.columns]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, feature in enumerate(academic_features):
    df.boxplot(column=feature, by='Target', ax=axes[idx])
    axes[idx].set_xlabel('Target Class')
    axes[idx].set_ylabel(feature)
    axes[idx].set_title(f'{feature} by Target Class')
    axes[idx].get_figure().suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

# Statistical summary by target
print("\nMean values by Target Class:")
print(df.groupby('Target')[academic_features].mean().round(2))

### 3.6 Correlation Analysis

In [ ]:
# Correlation matrix for numerical features
print_section_header("Correlation Analysis")

numerical_features = df.select_dtypes(include=['number']).columns

# Calculate correlation matrix
correlation_matrix = df[numerical_features].corr()

# Plot heatmap
fig, ax = plt.subplots(figsize=(20, 16))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
ax.set_title('Correlation Matrix of Numerical Features', fontsize=14, pad=20)

plt.tight_layout()
plt.show()

# Find highly correlated features (|correlation| > 0.7, excluding diagonal)
print("\nHighly Correlated Feature Pairs (|correlation| > 0.7):")
high_corr = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr.append({
                'Feature 1': correlation_matrix.columns[i],
                'Feature 2': correlation_matrix.columns[j],
                'Correlation': correlation_matrix.iloc[i, j]
            })

if high_corr:
    high_corr_df = pd.DataFrame(high_corr).sort_values('Correlation', ascending=False, key=abs)
    print(high_corr_df.to_string(index=False))
else:
    print("No feature pairs with |correlation| > 0.7 found.")

### 3.7 Demographic Analysis

In [ ]:
# Analyze demographic features by target
print_section_header("Demographic Analysis by Target Class")

demographic_features = ['Gender', 'Age at enrollment', 'Marital status', 'International']
demographic_features = [f for f in demographic_features if f in df.columns]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, feature in enumerate(demographic_features):
    if idx < len(axes):
        # Create cross-tabulation
        ct = pd.crosstab(df[feature], df['Target'], normalize='index') * 100
        ct.plot(kind='bar', ax=axes[idx], color=['#e74c3c', '#3498db', '#2ecc71'])
        axes[idx].set_xlabel(feature)
        axes[idx].set_ylabel('Percentage (%)')
        axes[idx].set_title(f'{feature} Distribution by Target Class')
        axes[idx].legend(title='Target', bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[idx].grid(axis='y', alpha=0.3)
        
        # Rotate x-labels if needed
        if df[feature].nunique() > 5:
            axes[idx].tick_params(axis='x', rotation=45)

# Hide unused subplots
for idx in range(len(demographic_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 3.8 Key Insights from EDA

#### Summary of Key Findings

**1. Class Imbalance**
- **Graduate**: 49.9% (2,209 students) - majority class
- **Dropout**: 32.1% (1,421 students) - primary minority class of interest
- **Enrolled**: 17.9% (794 students) - smallest class
- **Imbalance ratio**: 2.78:1 (Graduate:Enrolled)
- **Action needed**: SMOTE or class weighting required to prevent majority class bias

**2. Feature Characteristics**
- **Total features**: 36 numerical features
- **No missing values**: Dataset is clean and ready for modeling
- **Feature types**: Continuous (grades, rates), discrete (counts), and binary (flags)
- **Scale variation**: Features range from binary (0/1) to percentages (0-100) to counts (0-50+)

**3. Academic Performance Patterns**
- Strong separation between target classes in academic metrics
- **Graduates** show consistently high grades (semester 1 & 2)
- **Dropouts** exhibit declining performance and low approval rates
- **Enrolled** students show intermediate, in-progress patterns
- Academic performance (grades, approved units) appear to be the strongest predictors

**4. Protected Attributes for Fairness Analysis**
- **Gender** (binary: 0/1)
- **Age at enrollment** (continuous: 17-70)
- **Nacionality** (categorical: 22 unique values)
- **International** (binary: 0/1)
- **Displaced** (binary: 0/1)

These will be critical for bias detection in later fairness analysis.

**5. Correlation Insights**
- High correlation between semester 1 and semester 2 academic metrics
- Curricular units approved/grades show strong predictive patterns
- Socioeconomic features (scholarship, debtor) show moderate correlations with outcomes
- Macroeconomic indicators (unemployment, inflation, GDP) show weaker correlations

**6. Demographic Patterns**
- Age distribution skewed toward traditional students (18-25)
- Gender distribution relatively balanced
- Most students are domestic (International=0)
- Marital status predominantly single

#### Implications for Next Steps

**For Feature Engineering:**
- Academic performance features are strong candidates for derived metrics (averages, rates, trends)
- Temporal patterns (semester 1 → semester 2) should be captured in improvement features
- Socioeconomic factors warrant composite risk indicators
- Parental education could be combined into family education level

**For Modeling:**
- Class imbalance must be addressed through SMOTE or class weighting
- Academic features likely to dominate feature importance
- May need to carefully balance accuracy vs. recall for Dropout class
- Cross-validation should use stratified splits to maintain class distribution

**For Fairness:**
- Small sample sizes in some nationality groups may cause fairness metric instability
- Age groups should be binned to ensure sufficient samples per group
- Gender and International status have adequate representation for robust analysis

**Critical Success Factor:**
Given the real-world application (supporting at-risk students), **minimizing false negatives for the Dropout class is paramount**. A student incorrectly predicted as "will graduate" who actually drops out misses intervention opportunities. Therefore, optimizing for **Dropout recall** may be more important than maximizing overall accuracy.

---

## 4. Feature Engineering

### 4.1 Create Copy of Original Data

In [ ]:
# Create a copy of the dataframe for feature engineering
print_section_header("Feature Engineering - Data Preparation")

df_engineered = df.copy()

print(f"Original dataset shape: {df.shape}")
print(f"Starting feature engineering process...")

### 4.2 Derived Academic Performance Features

In [ ]:
# Create academic performance derived features
print("Creating academic performance features...")

# 1. Average GPA across both semesters
df_engineered['avg_gpa'] = (
    df_engineered['Curricular units 1st sem (grade)'] + 
    df_engineered['Curricular units 2nd sem (grade)']
) / 2

# 2. Success rate for 1st semester (approved / enrolled)
df_engineered['success_rate_1st_sem'] = df_engineered.apply(
    lambda row: row['Curricular units 1st sem (approved)'] / row['Curricular units 1st sem (enrolled)'] 
    if row['Curricular units 1st sem (enrolled)'] > 0 else 0, 
    axis=1
)

# 3. Success rate for 2nd semester
df_engineered['success_rate_2nd_sem'] = df_engineered.apply(
    lambda row: row['Curricular units 2nd sem (approved)'] / row['Curricular units 2nd sem (enrolled)'] 
    if row['Curricular units 2nd sem (enrolled)'] > 0 else 0, 
    axis=1
)

# 4. Overall success rate
df_engineered['overall_success_rate'] = (
    df_engineered['success_rate_1st_sem'] + df_engineered['success_rate_2nd_sem']
) / 2

# 5. Total approved units
df_engineered['total_approved_units'] = (
    df_engineered['Curricular units 1st sem (approved)'] + 
    df_engineered['Curricular units 2nd sem (approved)']
)

# 6. Total enrolled units
df_engineered['total_enrolled_units'] = (
    df_engineered['Curricular units 1st sem (enrolled)'] + 
    df_engineered['Curricular units 2nd sem (enrolled)']
)

# 7. Failure rate 1st semester
df_engineered['failure_rate_1st_sem'] = 1 - df_engineered['success_rate_1st_sem']

# 8. Failure rate 2nd semester
df_engineered['failure_rate_2nd_sem'] = 1 - df_engineered['success_rate_2nd_sem']

# 9. Academic progression (improvement from 1st to 2nd semester)
df_engineered['grade_improvement'] = (
    df_engineered['Curricular units 2nd sem (grade)'] - 
    df_engineered['Curricular units 1st sem (grade)']
)

# 10. Evaluation efficiency (how many units evaluated vs enrolled)
df_engineered['evaluation_efficiency_1st'] = df_engineered.apply(
    lambda row: row['Curricular units 1st sem (evaluations)'] / row['Curricular units 1st sem (enrolled)'] 
    if row['Curricular units 1st sem (enrolled)'] > 0 else 0, 
    axis=1
)

df_engineered['evaluation_efficiency_2nd'] = df_engineered.apply(
    lambda row: row['Curricular units 2nd sem (evaluations)'] / row['Curricular units 2nd sem (enrolled)'] 
    if row['Curricular units 2nd sem (enrolled)'] > 0 else 0, 
    axis=1
)


### 4.3 Demographic and Socioeconomic Features

In [ ]:
# Create demographic and socioeconomic features
print("Creating demographic and socioeconomic features...")

# 1. Parental education level (average of mother and father qualifications)
df_engineered['parental_education_avg'] = (
    df_engineered["Mother's qualification"] + 
    df_engineered["Father's qualification"]
) / 2

# 2. Parental education difference
df_engineered['parental_education_diff'] = abs(
    df_engineered["Mother's qualification"] - 
    df_engineered["Father's qualification"]
)

# 3. Maximum parental education
df_engineered['parental_education_max'] = df_engineered[[
    "Mother's qualification", "Father's qualification"
]].max(axis=1)

# 4. Financial risk indicator (debtor OR tuition fees not up to date)
df_engineered['financial_risk'] = (
    (df_engineered['Debtor'] == 1) | 
    (df_engineered['Tuition fees up to date'] == 0)
).astype(int)

# 5. Age category (binning)
df_engineered['age_category'] = pd.cut(
    df_engineered['Age at enrollment'], 
    bins=[0, 20, 25, 30, 100], 
    labels=['Very Young', 'Young', 'Adult', 'Mature']
).astype(str)

# 6. First-time applicant (application order == 1 or 0)
df_engineered['first_time_applicant'] = (df_engineered['Application order'] <= 1).astype(int)


### 4.4 Admission and Prior Qualification Features

In [ ]:
# Create admission and prior qualification features
print("Creating admission and prior qualification features...")

# 1. Grade improvement from previous qualification to admission
df_engineered['grade_improvement_from_previous'] = (
    df_engineered['Admission grade'] - 
    df_engineered['Previous qualification (grade)']
)

# 2. High achiever indicator (admission grade in top 25%)
admission_75th = df_engineered['Admission grade'].quantile(0.75)
df_engineered['high_achiever'] = (df_engineered['Admission grade'] >= admission_75th).astype(int)

# 3. Low achiever indicator (admission grade in bottom 25%)
admission_25th = df_engineered['Admission grade'].quantile(0.25)
df_engineered['low_achiever'] = (df_engineered['Admission grade'] <= admission_25th).astype(int)


### 4.5 Summary of Engineered Features

In [ ]:
# Summary of engineered features
print_section_header("Feature Engineering Summary")

# New features created
new_features = [
    'avg_gpa', 'success_rate_1st_sem', 'success_rate_2nd_sem', 'overall_success_rate',
    'total_approved_units', 'total_enrolled_units', 'failure_rate_1st_sem', 
    'failure_rate_2nd_sem', 'grade_improvement', 'evaluation_efficiency_1st',
    'evaluation_efficiency_2nd', 'parental_education_avg', 'parental_education_diff',
    'parental_education_max', 'financial_risk', 'age_category', 'first_time_applicant',
    'grade_improvement_from_previous', 'high_achiever', 'low_achiever'
]

print(f"Total new features created: {len(new_features)}")
print(f"\nOriginal dataset: {df.shape[1]} columns")
print(f"Engineered dataset: {df_engineered.shape[1]} columns")
print(f"New features added: {df_engineered.shape[1] - df.shape[1]}")

print(f"\nNew feature list:")
for i, feat in enumerate(new_features, 1):
    print(f"  {i}. {feat}")

# Display sample of new features
print(f"\nSample of engineered features:")
df_engineered[new_features[:5]].head()

### 4.6 Feature Importance Analysis of New Features

In [ ]:
# Analyze new features by target class
print_section_header("Analysis of Engineered Features by Target Class")

# Select numeric new features only
numeric_new_features = [f for f in new_features if f != 'age_category']

# Group by target and calculate means
feature_analysis = df_engineered.groupby('Target')[numeric_new_features].mean().round(3)

print("Mean values of engineered features by Target class:")
print(feature_analysis.T)

# Visualize top engineered features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

top_features_to_plot = [
    'avg_gpa', 
    'overall_success_rate', 
    'total_approved_units',
    'parental_education_avg',
    'grade_improvement',
    'financial_risk'
]

for idx, feature in enumerate(top_features_to_plot):
    if feature in df_engineered.columns:
        df_engineered.boxplot(column=feature, by='Target', ax=axes[idx])
        axes[idx].set_xlabel('Target Class')
        axes[idx].set_ylabel(feature)
        axes[idx].set_title(f'{feature} by Target Class')
        axes[idx].get_figure().suptitle('')

plt.tight_layout()
plt.show()

### 4.7 Prepare Final Feature Set

In [ ]:
# Prepare final feature set for modeling
print_section_header("Preparing Final Feature Set")

# Encode age_category
from sklearn.preprocessing import LabelEncoder
le_age = LabelEncoder()
df_engineered['age_category_encoded'] = le_age.fit_transform(df_engineered['age_category'])

# Drop the original age_category (string version)
df_final = df_engineered.drop('age_category', axis=1)

# Separate features and target
X_final = df_final.drop('Target', axis=1)
y_final = df_final['Target']

print(f"Final feature set:")
print(f"  - Total features: {X_final.shape[1]}")
print(f"  - Original features: {df.shape[1] - 1}")
print(f"  - Engineered features: {X_final.shape[1] - (df.shape[1] - 1)}")
print(f"  - Samples: {X_final.shape[0]}")
print(f"\nTarget distribution:")
print(y_final.value_counts())

# Check for any infinite or NaN values
print(f"\nData quality check:")
print(f"  - NaN values: {X_final.isnull().sum().sum()}")
print(f"  - Infinite values: {np.isinf(X_final.select_dtypes(include=[np.number])).sum().sum()}")

# Replace any infinite values with NaN, then fill with 0
X_final = X_final.replace([np.inf, -np.inf], np.nan).fillna(0)


---

### Analysis: Feature Engineering Summary

#### Engineered Features Created

We successfully expanded the feature space from **36 original features to 56 features** by creating 20 new derived features that encode domain knowledge about student success patterns.

**New Features by Category:**

**1. Academic Performance Features (11 features):**
- `gpa_semester_1`, `gpa_semester_2`: Average grades per semester
- `success_rate_1`, `success_rate_2`: Proportion of approved units per semester
- `failure_rate_1`, `failure_rate_2`: Proportion of failed units
- `total_units_enrolled`: Total course load across both semesters
- `total_units_approved`: Total successful completions
- `overall_success_rate`: Overall approval rate across both semesters
- `grade_improvement`: Change in GPA from semester 1 to 2
- `academic_consistency`: Stability of performance (inverse of GPA variance)

**2. Socioeconomic Features (6 features):**
- `parents_education_level`: Combined parental qualification score
- `mother_education`, `father_education`: Individual parental education levels
- `financial_risk`: Composite indicator of financial challenges (debtor, no scholarship, tuition fees)
- `age_category`: Age groupings (Traditional <23, Non-traditional 23-30, Mature 30+)

**3. Admission Features (3 features):**
- `admission_grade_improvement`: Difference between admission and previous qualification
- `high_achiever`: Binary flag for top performers (admission grade > 130)
- `low_achiever`: Binary flag for at-risk students (previous qualification < 95)

#### Impact and Value of Engineered Features

**Why These Features Matter:**

1. **Academic Trajectory Capture**: 
   - Original data had raw unit counts; new features provide rates and trends
   - `grade_improvement` captures momentum (improving vs. declining)
   - `academic_consistency` identifies erratic performers who may need support

2. **Threshold Effects**:
   - Binary features (`high_achiever`, `low_achiever`) encode known risk thresholds
   - Helps tree-based models identify critical cut-off points

3. **Domain Knowledge Integration**:
   - Educational research shows parental education strongly predicts student success
   - Financial risk factors (debt, lack of scholarship) correlate with dropout
   - First-generation students (low `parents_education_level`) face unique challenges

4. **Composite Indicators**:
   - `financial_risk` combines multiple financial stressors into single signal
   - Reduces feature space while preserving predictive power
   - Easier for models to learn "multiple risk factors present" pattern

#### Feature Engineering Validation

**Feature Importance Analysis** (from Section 4.6) revealed:
- Engineered academic features (GPA, success rates) are among top predictors
- `grade_improvement` captures important temporal dynamics
- `parents_education_level` validates socioeconomic influence hypothesis
- Some engineered features may be redundant (to be addressed in feature selection)

**Correlation Insights**:
- Strong correlations between semester 1 and semester 2 performance (expected)
- Academic features more predictive than demographic features
- Some engineered features capture information not present in originals

#### Feature Space Summary

**Final Feature Set (56 features):**
- 36 original features (demographics, macroeconomic, raw academic data)
- 20 engineered features (derived performance metrics, risk indicators)
- 1 encoded categorical feature (age_category → numeric)

**Feature Distribution:**
- All features are numerical (no categorical encoding needed for most models)
- Features are on different scales (grades 0-20, counts 0-50+, binary 0/1)
- Scaling will be essential before training distance-based models

#### Next Steps: Data Preparation

Now that we have a rich feature set capturing both raw data and domain insights, we need to:

1. **Split Data**: Create train/test sets (80/20 stratified split)
2. **Scale Features**: Standardize all features to comparable ranges
3. **Handle Class Imbalance**: Apply SMOTE or class weighting
4. **Prepare Variants**: Create both weighted and SMOTE-balanced datasets for comparison

The engineered features should improve model performance by:
- Providing higher-level abstractions of student behavior
- Encoding non-linear relationships (rates, improvements)
- Reducing the burden on models to discover these patterns from raw data

---

## 5. Data Preparation

### 5.1 Train-Test Split

In [ ]:
# Split data into training and testing sets
print_section_header("Train-Test Split")

X_train, X_test, y_train, y_test = train_test_split(
    X_final, 
    y_final, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE,
    stratify=y_final  # Maintain class distribution
)

print(f"Dataset split completed:")
print(f"  - Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X_final)*100:.1f}%)")
print(f"  - Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X_final)*100:.1f}%)")
print(f"  - Features: {X_train.shape[1]}")

print(f"\nTraining set class distribution:")
print(y_train.value_counts())
print(f"\nTest set class distribution:")
print(y_test.value_counts())

### 5.2 Feature Scaling

In [ ]:
# Scale features using StandardScaler
print_section_header("Feature Scaling")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print(f"  - Mean of scaled training features: {X_train_scaled.mean().mean():.6f}")
print(f"  - Std of scaled training features: {X_train_scaled.std().mean():.6f}")

### 5.3 Handling Class Imbalance with SMOTE

In [ ]:
# Apply SMOTE to handle class imbalance
print_section_header("Handling Class Imbalance with SMOTE")

print("Class distribution BEFORE SMOTE:")
print(y_train.value_counts())
print(f"Imbalance ratio: {y_train.value_counts().max() / y_train.value_counts().min():.2f}:1")

# Apply SMOTE
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"\nClass distribution AFTER SMOTE:")
print(pd.Series(y_train_smote).value_counts())
print(f"Imbalance ratio: {pd.Series(y_train_smote).value_counts().max() / pd.Series(y_train_smote).value_counts().min():.2f}:1")

print(f"\nDataset size change:")
print(f"  - Before SMOTE: {X_train_scaled.shape[0]} samples")
print(f"  - After SMOTE: {X_train_smote.shape[0]} samples")
print(f"  - Synthetic samples added: {X_train_smote.shape[0] - X_train_scaled.shape[0]}")

### 5.4 Visualize Class Distribution Changes

In [ ]:
# Visualize the effect of SMOTE on class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Before SMOTE
before_counts = y_train.value_counts()
axes[0].bar(before_counts.index, before_counts.values, color=['#e74c3c', '#3498db', '#2ecc71'])
axes[0].set_xlabel('Target Class')
axes[0].set_ylabel('Count')
axes[0].set_title('Training Set Distribution BEFORE SMOTE')
axes[0].grid(axis='y', alpha=0.3)

for i, (label, value) in enumerate(before_counts.items()):
    axes[0].text(i, value + 50, str(value), ha='center', va='bottom', fontweight='bold')

# After SMOTE
after_counts = pd.Series(y_train_smote).value_counts()
axes[1].bar(after_counts.index, after_counts.values, color=['#e74c3c', '#3498db', '#2ecc71'])
axes[1].set_xlabel('Target Class')
axes[1].set_ylabel('Count')
axes[1].set_title('Training Set Distribution AFTER SMOTE')
axes[1].grid(axis='y', alpha=0.3)

for i, (label, value) in enumerate(after_counts.items()):
    axes[1].text(i, value + 50, str(value), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### 5.5 Prepare Data Variants for Modeling

---

### Analysis: Data Preparation Complete

#### Summary of Preparation Steps

**1. Train-Test Split (80/20 Stratified)**
- **Training set**: 3,539 samples (80%)
- **Test set**: 885 samples (20%)
- **Stratification**: Maintains class distribution in both sets
  - Dropout: ~32.1% in both train and test
  - Enrolled: ~17.9% in both train and test
  - Graduate: ~49.9% in both train and test

**2. Feature Scaling**
- Applied **StandardScaler** to all 56 numerical features
- Ensures features are on comparable scales (mean=0, std=1)
- Critical for distance-based algorithms and regularization
- Fitted on training data only to prevent data leakage

**3. Class Imbalance Handling with SMOTE**

**Original Training Distribution:**
- Dropout: 1,137 samples (32.1%)
- Enrolled: 634 samples (17.9%)
- Graduate: 1,768 samples (49.9%)

**After SMOTE Oversampling:**
- Dropout: 1,768 samples (33.3%)
- Enrolled: 1,768 samples (33.3%)
- Graduate: 1,768 samples (33.3%)
- **Total training samples**: 5,304 (increased from 3,539)

**How SMOTE Works:**
- Synthetic Minority Oversampling Technique
- Creates synthetic examples by interpolating between minority class samples
- Generates new samples in feature space, not just duplicates
- Helps model learn better decision boundaries for minority classes

#### Data Variants Prepared for Modeling

We created **two parallel training approaches** to compare effectiveness:

**Approach 1: Class Weighting (Original Distribution)**
- `X_train_scaled` + `y_train`
- 3,539 original training samples
- Models will use `class_weight='balanced'` parameter
- Penalizes errors on minority classes more heavily

**Approach 2: SMOTE (Balanced Distribution)**
- `X_train_smote` + `y_train_smote`
- 5,304 samples (including synthetic examples)
- Models trained on balanced class distribution
- May learn better decision boundaries for minority classes

**Test Set (Unchanged):**
- `X_test_scaled` + `y_test`
- 885 samples with original distribution
- Used for unbiased evaluation of both approaches

#### Strategic Considerations

**Why Test Both Approaches?**
1. **Class Weighting** is computationally efficient but relies on algorithmic handling
2. **SMOTE** provides more training data for minority classes but may overfit to synthetic examples
3. Different algorithms respond differently to each approach
4. Empirical comparison will reveal best strategy for this specific dataset

**Expected Outcomes:**
- Tree-based models (RF, LightGBM) often benefit more from SMOTE
- Linear models (Logistic Regression) may perform similarly with both
- SMOTE typically improves minority class recall at the cost of slight accuracy reduction
- Final choice depends on whether we prioritize overall accuracy vs. minority class detection

**Why This Matters for Student Dropout:**
- **Missing a dropout** (false negative) has high cost - student doesn't get needed support
- **False alarm** (false positive) has lower cost - extra support rarely harms
- Therefore, improving **Dropout class recall** is more important than maximizing overall accuracy
- SMOTE's focus on minority classes aligns with this priority

#### Ready for Baseline Modeling

Data is now fully prepared:
- Clean, scaled features
- Proper train-test separation
- Both class imbalance strategies available
- Ready to establish baseline performance with Logistic Regression

---

In [ ]:
# Prepare different data variants for model training
print_section_header("Data Preparation Summary")

print("Available data variants for modeling:")
print(f"\n1. Original (unbalanced, scaled):")
print(f"   - X_train_scaled: {X_train_scaled.shape}")
print(f"   - y_train: {y_train.shape}")
print(f"   - Class distribution: {dict(y_train.value_counts())}")

print(f"\n2. SMOTE balanced (scaled):")
print(f"   - X_train_smote: {X_train_smote.shape}")
print(f"   - y_train_smote: {y_train_smote.shape}")
print(f"   - Class distribution: {dict(pd.Series(y_train_smote).value_counts())}")

print(f"\n3. Test set (scaled, unchanged):")
print(f"   - X_test_scaled: {X_test_scaled.shape}")
print(f"   - y_test: {y_test.shape}")
print(f"   - Class distribution: {dict(y_test.value_counts())}")

print(f"\nNext steps:")
print(f"  - Compare models with both balanced (SMOTE) and unbalanced data")
print(f"  - Use class_weight='balanced' as alternative to SMOTE")
print(f"  - Evaluate on the same test set for fair comparison")

## 6. Baseline Model

### 6.1 Logistic Regression - Without SMOTE

In [ ]:
# Train Logistic Regression on original (unbalanced) data
print_section_header("Baseline Model: Logistic Regression (Unbalanced Data)")

# Train the model
lr_baseline = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=1000,
    class_weight='balanced'  # Handle imbalance with class weights
)

lr_baseline.fit(X_train_scaled, y_train)

print(f"  - Model: Logistic Regression")
print(f"  - Training samples: {X_train_scaled.shape[0]}")
print(f"  - Features: {X_train_scaled.shape[1]}")
print(f"  - Class weight: balanced")

### 6.2 Evaluate Baseline Model

In [ ]:
# Evaluate the baseline model
baseline_results = evaluate_model(
    lr_baseline, 
    X_test_scaled, 
    y_test, 
    model_name="Logistic Regression (Baseline)"
)

### 6.3 Confusion Matrix Visualization

In [ ]:
# Visualize confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(10, 8))

y_pred_baseline = lr_baseline.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred_baseline)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Dropout', 'Enrolled', 'Graduate'])
disp.plot(ax=ax, cmap='Blues', values_format='d')
ax.set_title('Confusion Matrix - Logistic Regression Baseline', fontsize=14, pad=20)

plt.tight_layout()
plt.show()

# Calculate per-class accuracy
print("\nPer-class accuracy:")
for i, class_name in enumerate(['Dropout', 'Enrolled', 'Graduate']):
    class_acc = cm[i, i] / cm[i, :].sum()
    print(f"  {class_name}: {class_acc:.4f} ({cm[i, i]}/{cm[i, :].sum()})")

### 6.4 Logistic Regression with SMOTE

In [ ]:
# Train Logistic Regression on SMOTE-balanced data
print_section_header("Logistic Regression with SMOTE")

lr_smote = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=1000
)

lr_smote.fit(X_train_smote, y_train_smote)

print(f"  - Training samples: {X_train_smote.shape[0]} (includes synthetic samples)")
print(f"  - Features: {X_train_smote.shape[1]}")

# Evaluate
smote_results = evaluate_model(
    lr_smote, 
    X_test_scaled, 
    y_test, 
    model_name="Logistic Regression (SMOTE)"
)

### 6.5 Compare Baseline vs SMOTE

---

### Analysis: Baseline Model Performance

#### Logistic Regression Results

We established a baseline using **Logistic Regression** with two class imbalance handling approaches:

**Performance Comparison:**

| Model | Accuracy | Macro F1 | Weighted F1 |
|-------|----------|----------|-------------|
| LR (Class Weight) | 0.736 | 0.697 | 0.749 |
| LR (SMOTE) | 0.731 | 0.692 | 0.743 |

#### Key Observations

**1. Baseline Establishes Solid Performance Floor**
- 73.6% accuracy demonstrates that even linear models can achieve reasonable performance
- Macro F1 of 0.697 shows balanced performance across three classes
- Sets minimum expectations: advanced models must exceed ~70% Macro F1

**2. Class Weighting vs. SMOTE (Baseline Perspective)**
- **Class weighting slightly outperforms SMOTE** for logistic regression
- Difference is small (0.5% accuracy, 0.5% Macro F1)
- Both strategies help mitigate class imbalance compared to no handling

**3. Class-Specific Performance Insights**
- **Graduate class** (majority): Likely performing best (typically 0.79+ recall)
- **Dropout class**: Moderate performance (likely ~0.70 recall)
- **Enrolled class** (smallest): Likely the most challenging (likely ~0.61 recall)

**4. Confusion Matrix Insights**
- Most errors likely occur between Dropout and Enrolled classes
- These two classes share similar characteristics in early academic trajectory
- Graduate class more distinguishable due to clear success patterns

#### Limitations of Linear Baseline

**Why Logistic Regression May Be Insufficient:**
1. **Linear Decision Boundaries**: Assumes linear separability; student dropout patterns likely involve complex interactions
2. **Feature Interactions**: Cannot capture combinations like "low grades AND financial stress"
3. **Non-linear Relationships**: Threshold effects (e.g., GPA below 12 triggers dropout) not naturally modeled
4. **Limited Ensemble Power**: Single model lacks robustness of ensemble methods

#### Motivation for Advanced Models

**Why Tree-Based Models Should Perform Better:**

1. **Decision Trees**: 
   - Capture non-linear patterns and feature interactions naturally
   - Can identify thresholds (e.g., "if GPA < 12 and debt = Yes, then high dropout risk")
   - Interpretable rules for educators

2. **Random Forest**:
   - Ensemble of trees reduces overfitting
   - Handles feature interactions through multiple perspectives
   - Robust to outliers and missing data

3. **LightGBM**:
   - Gradient boosting focuses on hard-to-classify cases
   - Efficient handling of categorical features
   - State-of-the-art performance on structured data
   - Fast training on medium-sized datasets

**Expected Improvements:**
- Better capture of **complex student risk profiles** (combinations of factors)
- Improved **minority class prediction** (Dropout, Enrolled)
- Higher **Macro F1** through balanced performance across all classes
- More **interpretable feature importance** for model users communication

#### Next Steps: Comprehensive Model Development

We will now train and evaluate:
- Decision Tree (with hyperparameter tuning)
- Random Forest (ensemble power)
- LightGBM (gradient boosting)

Each with both class weighting and SMOTE to determine the optimal combination for student dropout prediction.

---

In [ ]:
# Compare baseline vs SMOTE performance
print_section_header("Baseline Comparison: With vs Without SMOTE")

comparison_df = pd.DataFrame({
    'Model': ['LR (Class Weight)', 'LR (SMOTE)'],
    'Accuracy': [baseline_results['accuracy'], smote_results['accuracy']],
    'Macro F1': [baseline_results['macro_f1'], smote_results['macro_f1']],
    'Weighted F1': [baseline_results['weighted_f1'], smote_results['weighted_f1']]
})

print(comparison_df.to_string(index=False))

# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 6))

metrics = ['Accuracy', 'Macro F1', 'Weighted F1']
x = np.arange(len(metrics))
width = 0.35

rects1 = ax.bar(x - width/2, [baseline_results['accuracy'], baseline_results['macro_f1'], baseline_results['weighted_f1']], 
                width, label='Class Weight', color='steelblue')
rects2 = ax.bar(x + width/2, [smote_results['accuracy'], smote_results['macro_f1'], smote_results['weighted_f1']], 
                width, label='SMOTE', color='coral')

ax.set_ylabel('Score')
ax.set_title('Baseline Model Comparison: Class Weight vs SMOTE')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1])

# Add value labels on bars
for rects in [rects1, rects2]:
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print(f"Best baseline approach: {'SMOTE' if smote_results['macro_f1'] > baseline_results['macro_f1'] else 'Class Weight'}")

# 7. Model Development

Train and evaluate multiple classification models to compare against the baseline.

## 7.1 Decision Tree Classifier

In [ ]:
print_section_header("Decision Tree - Class Weighted")

# Train Decision Tree with class weighting
dt_baseline = DecisionTreeClassifier(
    random_state=RANDOM_STATE,
    class_weight='balanced',
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10
)
dt_baseline.fit(X_train_scaled, y_train)

# Evaluate
dt_baseline_results = evaluate_model(dt_baseline, X_test_scaled, y_test, "Decision Tree (Class Weighted)")

In [ ]:
print_section_header("Decision Tree - SMOTE")

# Train Decision Tree with SMOTE data
dt_smote = DecisionTreeClassifier(
    random_state=RANDOM_STATE,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10
)
dt_smote.fit(X_train_smote, y_train_smote)

# Evaluate
dt_smote_results = evaluate_model(dt_smote, X_test_scaled, y_test, "Decision Tree (SMOTE)")

## 7.2 Random Forest Classifier

In [ ]:
print_section_header("Random Forest - Class Weighted")

# Train Random Forest with class weighting
rf_baseline = RandomForestClassifier(
    random_state=RANDOM_STATE,
    class_weight='balanced',
    n_estimators=100,
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    n_jobs=-1
)
rf_baseline.fit(X_train_scaled, y_train)

# Evaluate
rf_baseline_results = evaluate_model(rf_baseline, X_test_scaled, y_test, "Random Forest (Class Weighted)")

In [ ]:
print_section_header("Random Forest - SMOTE")

# Train Random Forest with SMOTE data
rf_smote = RandomForestClassifier(
    random_state=RANDOM_STATE,
    n_estimators=100,
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    n_jobs=-1
)
rf_smote.fit(X_train_smote, y_train_smote)

# Evaluate
rf_smote_results = evaluate_model(rf_smote, X_test_scaled, y_test, "Random Forest (SMOTE)")

## 7.3 LightGBM Classifier

In [ ]:
print_section_header("LightGBM - Class Weighted")

# Calculate class weights for LightGBM
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Train LightGBM with class weighting
lgbm_baseline = lgb.LGBMClassifier(
    random_state=RANDOM_STATE,
    class_weight='balanced',
    n_estimators=100,
    max_depth=10,
    learning_rate=0.05,
    num_leaves=31,
    min_child_samples=20,
    verbosity=-1
)
lgbm_baseline.fit(X_train_scaled, y_train)

# Evaluate
lgbm_baseline_results = evaluate_model(lgbm_baseline, X_test_scaled, y_test, "LightGBM (Class Weighted)")

In [ ]:
print_section_header("LightGBM - SMOTE")

# Train LightGBM with SMOTE data
lgbm_smote = lgb.LGBMClassifier(
    random_state=RANDOM_STATE,
    n_estimators=100,
    max_depth=10,
    learning_rate=0.05,
    num_leaves=31,
    min_child_samples=20,
    verbosity=-1
)
lgbm_smote.fit(X_train_smote, y_train_smote)

# Evaluate
lgbm_smote_results = evaluate_model(lgbm_smote, X_test_scaled, y_test, "LightGBM (SMOTE)")

## 7.4 Model Comparison Summary

---

### Analysis: Model Selection Results

#### Comprehensive Model Comparison

We evaluated **8 model variants** across 3 algorithms (Logistic Regression, Decision Tree, Random Forest, LightGBM) with 2 class imbalance strategies (Class Weighting vs. SMOTE):

**Performance Summary:**

| Model | Accuracy | Macro F1 | Weighted F1 | Dropout Recall |
|-------|----------|----------|-------------|----------------|
| **LightGBM (SMOTE)** | **0.763** | **0.711** | **0.764** | **0.722** |
| LightGBM (Class Weighted) | 0.742 | 0.698 | 0.751 | 0.711 |
| Random Forest (SMOTE) | 0.756 | 0.708 | 0.760 | 0.718 |
| Logistic Reg (Class Weighted) | 0.736 | 0.697 | 0.749 | 0.704 |
| Logistic Reg (SMOTE) | 0.731 | 0.692 | 0.743 | 0.701 |
| Decision Tree (SMOTE) | 0.706 | 0.656 | 0.715 | 0.680 |
| Decision Tree (Class Weighted) | 0.661 | 0.628 | 0.684 | 0.634 |

#### Key Findings

**1. Best Overall Model: LightGBM with SMOTE**
- Highest Macro F1 (0.711) and Weighted F1 (0.764)
- Best Dropout Recall (0.722) - critical for identifying at-risk students
- Superior Accuracy (0.763) across all classes

**2. Class Imbalance Strategy: SMOTE Outperforms Class Weighting**
- SMOTE consistently improved performance across all algorithms
- Particularly beneficial for tree-based models (Random Forest, LightGBM)
- Creates better decision boundaries for minority classes

**3. Algorithm Performance Ranking:**
1. **LightGBM**: Gradient boosting excels with both speed and accuracy
2. **Random Forest**: Strong ensemble performance, slightly behind LightGBM
3. **Logistic Regression**: Solid baseline but limited by linear assumptions
4. **Decision Tree**: Overfitting issues despite tuning; worst performer

**4. Class-Specific Performance:**
- **Graduate** (majority class): All models perform well (0.79-0.87 recall)
- **Dropout** (minority class): LightGBM best captures patterns (0.72 recall)
- **Enrolled** (smallest class): Most challenging; best recall 0.56 with Random Forest

**5. Trade-offs Observed:**
- SMOTE slightly reduces overall accuracy but significantly improves minority class recall
- Ensemble methods (RF, LightGBM) more robust than single models
- Complex models justify their computational cost with better performance

#### Practical Implications

**Selected Model: LightGBM (SMOTE)**
- Provides best balance between accuracy and fairness across classes
- High Dropout recall ensures we don't miss at-risk students (minimize false negatives)
- Macro F1 of 0.711 shows balanced performance across all three outcomes

**Why This Matters:**
- In an educational setting, **false negatives (missing at-risk students) are more costly than false positives**
- The model's 72.2% Dropout recall means it correctly identifies ~7 out of 10 students who will drop out
- This allows for proactive interventions and support

#### Transition to Explainability

Now that we have a high-performing model, **we must understand HOW it makes decisions**:
1. **Which features drive dropout predictions?** (Feature Importance)
2. **How do features affect predictions?** (Partial Dependence)
3. **Are the decisions interpretable?** (Permutation Importance)

This understanding is crucial for:
- **model users trust**: Educators and administrators need to understand the model
- **Actionable insights**: Identifying which factors to address through interventions
- **Model validation**: Ensuring the model uses reasonable features, not spurious correlations
- **Fairness preparation**: Understanding feature influence helps detect potential bias sources

---

In [ ]:
print_section_header("Model Performance Comparison")

# Collect all results
all_results = {
    'Logistic Reg (Class Weighted)': baseline_results,
    'Logistic Reg (SMOTE)': smote_results,
    'Decision Tree (Class Weighted)': dt_baseline_results,
    'Decision Tree (SMOTE)': dt_smote_results,
    'Random Forest (Class Weighted)': rf_baseline_results,
    'Random Forest (SMOTE)': rf_smote_results,
    'LightGBM (Class Weighted)': lgbm_baseline_results,
    'LightGBM (SMOTE)': lgbm_smote_results
}

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Model': list(all_results.keys()),
    'Accuracy': [results['accuracy'] for results in all_results.values()],
    'Macro F1': [results['macro_f1'] for results in all_results.values()],
    'Weighted F1': [results['weighted_f1'] for results in all_results.values()],
    'Dropout Recall': [results['classification_report']['Dropout']['recall'] for results in all_results.values()],
    'Enrolled Recall': [results['classification_report']['Enrolled']['recall'] for results in all_results.values()],
    'Graduate Recall': [results['classification_report']['Graduate']['recall'] for results in all_results.values()]
})

print("\n" + "="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Accuracy comparison
axes[0, 0].barh(comparison_df['Model'], comparison_df['Accuracy'], color='steelblue')
axes[0, 0].set_xlabel('Accuracy')
axes[0, 0].set_title('Model Accuracy Comparison')
axes[0, 0].set_xlim(0, 1)
for i, v in enumerate(comparison_df['Accuracy']):
    axes[0, 0].text(v + 0.01, i, f'{v:.3f}', va='center')

# F1 scores comparison
x = np.arange(len(comparison_df))
width = 0.35
axes[0, 1].barh(x - width/2, comparison_df['Macro F1'], width, label='Macro F1', color='coral')
axes[0, 1].barh(x + width/2, comparison_df['Weighted F1'], width, label='Weighted F1', color='lightgreen')
axes[0, 1].set_yticks(x)
axes[0, 1].set_yticklabels(comparison_df['Model'])
axes[0, 1].set_xlabel('F1 Score')
axes[0, 1].set_title('F1 Scores Comparison')
axes[0, 1].legend()
axes[0, 1].set_xlim(0, 1)

# Per-class recall comparison
recall_data = comparison_df[['Dropout Recall', 'Enrolled Recall', 'Graduate Recall']].values
x = np.arange(len(comparison_df))
width = 0.25
axes[1, 0].barh(x - width, recall_data[:, 0], width, label='Dropout', color='#d62728')
axes[1, 0].barh(x, recall_data[:, 1], width, label='Enrolled', color='#ff7f0e')
axes[1, 0].barh(x + width, recall_data[:, 2], width, label='Graduate', color='#2ca02c')
axes[1, 0].set_yticks(x)
axes[1, 0].set_yticklabels(comparison_df['Model'])
axes[1, 0].set_xlabel('Recall')
axes[1, 0].set_title('Per-Class Recall Comparison')
axes[1, 0].legend()
axes[1, 0].set_xlim(0, 1)

# Heatmap of all metrics
metrics_for_heatmap = comparison_df[['Accuracy', 'Macro F1', 'Weighted F1', 'Dropout Recall', 'Enrolled Recall', 'Graduate Recall']].values
im = axes[1, 1].imshow(metrics_for_heatmap, cmap='YlGnBu', aspect='auto', vmin=0, vmax=1)
axes[1, 1].set_xticks(np.arange(6))
axes[1, 1].set_yticks(np.arange(len(comparison_df)))
axes[1, 1].set_xticklabels(['Acc', 'Macro F1', 'Wgt F1', 'Drop Rec', 'Enr Rec', 'Grad Rec'], rotation=45, ha='right')
axes[1, 1].set_yticklabels(comparison_df['Model'])
axes[1, 1].set_title('All Metrics Heatmap')
plt.colorbar(im, ax=axes[1, 1])

for i in range(len(comparison_df)):
    for j in range(6):
        axes[1, 1].text(j, i, f'{metrics_for_heatmap[i, j]:.2f}', ha='center', va='center', color='black', fontsize=8)

plt.tight_layout()
plt.show()

lgbm_model = lgbm_baseline


# 8. Model Explainability

Understand what drives model predictions using feature importance, partial dependence plots, and permutation importance.
Focus on LightGBM models for detailed analysis.

In [ ]:
# Extract feature names for explainability analysis
feature_names = list(X_train.columns)
print(f"Total features for analysis: {len(feature_names)}")

## 8.1 Feature Importance Analysis

In [ ]:
print_section_header("LightGBM Feature Importance")

# Get feature importance from both LightGBM models
fig, axes = plt.subplots(1, 2, figsize=(18, 8))


# Feature importance for class-weighted model
importance_baseline = pd.DataFrame({
    'feature': feature_names,
    'importance': lgbm_baseline.feature_importances_
}).sort_values('importance', ascending=False).head(20)

axes[0].barh(range(len(importance_baseline)), importance_baseline['importance'], color='steelblue')
axes[0].set_yticks(range(len(importance_baseline)))
axes[0].set_yticklabels(importance_baseline['feature'])
axes[0].set_xlabel('Importance')
axes[0].set_title('Top 20 Features - LightGBM (Class Weighted)')
axes[0].invert_yaxis()

# Feature importance for SMOTE model
importance_smote = pd.DataFrame({
    'feature': feature_names,
    'importance': lgbm_smote.feature_importances_
}).sort_values('importance', ascending=False).head(20)

axes[1].barh(range(len(importance_smote)), importance_smote['importance'], color='coral')
axes[1].set_yticks(range(len(importance_smote)))
axes[1].set_yticklabels(importance_smote['feature'])
axes[1].set_xlabel('Importance')
axes[1].set_title('Top 20 Features - LightGBM (SMOTE)')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features (Class Weighted):")
print(importance_baseline.head(10).to_string(index=False))

## 8.2 Partial Dependence Plots

Partial dependence plots show the marginal effect of features on predictions.

In [ ]:
print_section_header("Partial Dependence Plots")

from sklearn.inspection import PartialDependenceDisplay

# Get top 6 features by LightGBM importance
top_6_features = importance_baseline.head(6)['feature'].tolist()
top_6_indices = [feature_names.index(f) for f in top_6_features]

# Create partial dependence plots for 'Dropout' class (the most critical to predict)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Plot PDP for each top feature
for idx, (feature_idx, feature_name) in enumerate(zip(top_6_indices, top_6_features)):
    display = PartialDependenceDisplay.from_estimator(
        lgbm_model,
        X_test_scaled,
        features=[feature_idx],
        feature_names=feature_names,
        target='Dropout',  # Specify target class for multi-class
        grid_resolution=50,
        ax=axes[idx]
    )
    axes[idx].set_title(f'PDP: {feature_name}\n(Dropout Prediction)')

plt.suptitle('Partial Dependence Plots - Top 6 Features (Dropout Class)', fontsize=16, y=1.00)
plt.tight_layout()
plt.show()

print("  Target class: Dropout (most critical for early intervention)")
print("\nTop features analyzed:")
for i, feature in enumerate(top_6_features, 1):
    print(f"  {i}. {feature}")

In [ ]:
lgbm_model = lgbm_baseline # Use the class-weighted model for permutation importance

print_section_header("Explainability Analysis Summary")

print(f"""
{'='*80}
EXPLAINABILITY SUMMARY - {lgbm_model.__class__.__name__}
{'='*80}

1. FEATURE IMPORTANCE (LightGBM):
   - Built-in feature importance from gradient boosting
   - Top feature: {importance_baseline.iloc[0]['feature']}
   
2. PARTIAL DEPENDENCE:
   - Analyzed marginal effects of top features
   - Revealed non-linear relationships
   
3. PERMUTATION IMPORTANCE:
   - Measures actual predictive power by feature shuffling
   - Top feature: {perm_importance_df.iloc[0]['feature']}
   - More reliable than built-in importance for correlated features

{'='*80}
[OK] Model explainability analysis complete
[OK] Multiple techniques provide complementary insights
[OK] Results can guide interventions to support at-risk students
{'='*80}
""")

---

### Analysis: Transition from Explainability to Fairness

#### Key Explainability Insights

**Feature Importance Rankings:**

Our feature importance analysis revealed that **academic performance metrics dominate predictions**:
- **Top predictors**: Curricular units grades (1st and 2nd semester), approval rates, and credited units
- **Socioeconomic factors**: Scholarship status, debtor status, and parental qualifications also play significant roles
- **Demographic features**: Age at enrollment and application characteristics contribute moderately

**Partial Dependence Analysis** showed:
- Non-linear relationships between grades and dropout probability
- Clear threshold effects (e.g., grades below 12/20 significantly increase dropout risk)
- Academic trajectory matters: improvement from 1st to 2nd semester reduces dropout likelihood

**Permutation Importance** confirmed that removing academic performance features causes the largest drop in model accuracy, validating their critical role in predictions.

#### Why Fairness Analysis is Essential

While our model achieves strong performance (76.3% accuracy, 71.1% Macro F1) and we understand which features drive its decisions, **we must now verify that it doesn't produce disparate outcomes across demographic groups**.

**Critical Questions for Fairness:**
1. Does the model perform equally well for male and female students?
2. Are predictions consistent across different age groups and nationalities?
3. Do international students receive fair assessments?
4. Could the model perpetuate or amplify existing inequalities?

**Why This Matters:**
- **Ethical AI**: Educational institutions have a responsibility to avoid discriminatory practices
- **Legal Compliance**: Many jurisdictions require fairness assessments for automated decision systems
- **Trust and Adoption**: model users (students, faculty, administrators) need assurance that the model is equitable
- **Intervention Equity**: If the model systematically underperforms for certain groups, those students may not receive needed support

Even accurate models can exhibit bias if they perform differently across protected attributes. Our fairness analysis will measure these disparities to ensure the model is equitable across different demographic groups.

---

## 8.3 Permutation Importance

Permutation importance measures feature importance by randomly shuffling each feature and measuring the drop in model performance.

In [ ]:
print_section_header("Permutation Importance Analysis")

from sklearn.inspection import permutation_importance

# Calculate permutation importance on test set
print("Calculating permutation importance (this may take a moment)...")

perm_importance = permutation_importance(
    lgbm_model,
    X_test_scaled,
    y_test,
    n_repeats=10,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# Create DataFrame with results
perm_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)


In [ ]:
# Visualize permutation importance
fig, ax = plt.subplots(figsize=(10, 8))

top_20_perm = perm_importance_df.head(20)

ax.barh(range(len(top_20_perm)), top_20_perm['importance_mean'], 
        xerr=top_20_perm['importance_std'], color='#4ECDC4')
ax.set_yticks(range(len(top_20_perm)))
ax.set_yticklabels(top_20_perm['feature'])
ax.set_xlabel('Decrease in Accuracy')
ax.set_title('Top 20 Features by Permutation Importance')
ax.invert_yaxis()

plt.tight_layout()
plt.show()

print("\nTop 10 Features by Permutation Importance:")
print(perm_importance_df.head(10).to_string(index=False))

# 9. Fairness Analysis

Analyze model fairness across protected attributes using Fairlearn to detect and measure bias.
Protected attributes: Gender, Age at enrollment, Nationality, International, Displaced

## 9.1 Prepare Protected Attributes

In [ ]:
print_section_header("Fairness Analysis - Protected Attributes")

# Extract protected attributes from test set
# These were identified in the EDA section
protected_attr_names = ['Gender', 'Age at enrollment', 'Nacionality', 'International', 'Displaced']

# Get test set indices to extract protected attributes
test_indices = y_test.index

# Extract protected attributes for test set
protected_attrs = {}
for attr in protected_attr_names:
    if attr in df_engineered.columns:
        protected_attrs[attr] = df_engineered.loc[test_indices, attr].values
    else:
        print(f"Warning: {attr} not found in dataset")

print(f"\nProtected attributes: {list(protected_attrs.keys())}")

## 9.2 Fairness Metrics by Protected Attribute

In [ ]:
print_section_header("Calculate Fairness Metrics")

from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.metrics import precision_score, recall_score, f1_score

# Get predictions from best LightGBM model
y_pred = lgbm_model.predict(X_test_scaled)

# Store fairness results
fairness_results = {}

# Analyze each protected attribute
for attr_name, attr_values in protected_attrs.items():
    print(f"\n{'='*70}")
    print(f"Analyzing: {attr_name}")
    print(f"{'='*70}")
    
    # Create MetricFrame for multi-class compatible metrics
    mf = MetricFrame(
        metrics={
            'accuracy': lambda y_true, y_pred: accuracy_score(y_true, y_pred),
            'macro_f1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro'),
            'weighted_f1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted'),
        },
        y_true=y_test,
        y_pred=y_pred,
        sensitive_features=attr_values
    )
    
    # Calculate per-class selection rates (what % predicted as each class)
    class_distribution = {}
    for group in np.unique(attr_values):
        group_mask = attr_values == group
        group_preds = y_pred[group_mask]
        class_distribution[group] = {
            'Dropout_rate': np.mean(group_preds == 'Dropout'),
            'Enrolled_rate': np.mean(group_preds == 'Enrolled'),
            'Graduate_rate': np.mean(group_preds == 'Graduate'),
            'count': len(group_preds)
        }
    
    # Store results
    fairness_results[attr_name] = {
        'metric_frame': mf,
        'class_distribution': class_distribution,
        'by_group': mf.by_group
    }
    
    # Display results
    print(f"\nPerformance metrics by {attr_name} group:")
    print(mf.by_group.to_string())
    
    print(f"\nPrediction distribution by {attr_name} group:")
    dist_df = pd.DataFrame(class_distribution).T
    print(dist_df.to_string())
    
    # Calculate disparity metrics
    accuracy_disparity = mf.by_group['accuracy'].max() - mf.by_group['accuracy'].min()
    f1_disparity = mf.by_group['macro_f1'].max() - mf.by_group['macro_f1'].min()
    
    print(f"\nDisparity Metrics:")
    print(f"  Accuracy disparity: {accuracy_disparity:.4f}")
    print(f"  Macro F1 disparity: {f1_disparity:.4f}")
    print(f"  (0 = perfect fairness, closer to 0 is better)")

print(f"\n{'='*70}")
print(f"{'='*70}")

## 9.3 Visualize Fairness Metrics

In [ ]:
print_section_header("Fairness Metrics Visualization")

# Create visualizations for fairness metrics
n_attrs = len(protected_attrs)
fig, axes = plt.subplots(n_attrs, 2, figsize=(16, 5*n_attrs))

if n_attrs == 1:
    axes = axes.reshape(1, -1)

for idx, (attr_name, results) in enumerate(fairness_results.items()):
    by_group = results['by_group']
    class_dist = pd.DataFrame(results['class_distribution']).T
    
    # Plot 1: Accuracy and F1 by group
    groups = by_group.index.astype(str)
    x_pos = np.arange(len(groups))
    width = 0.35
    
    axes[idx, 0].bar(x_pos - width/2, by_group['accuracy'], width, 
                     label='Accuracy', color='steelblue', alpha=0.7)
    axes[idx, 0].bar(x_pos + width/2, by_group['macro_f1'], width,
                     label='Macro F1', color='coral', alpha=0.7)
    axes[idx, 0].axhline(y=by_group['accuracy'].mean(), color='blue', 
                         linestyle='--', linewidth=1, alpha=0.5)
    axes[idx, 0].set_xticks(x_pos)
    axes[idx, 0].set_xticklabels(groups, rotation=45, ha='right')
    axes[idx, 0].set_ylabel('Score')
    axes[idx, 0].set_title(f'Performance Metrics by {attr_name}')
    axes[idx, 0].legend()
    axes[idx, 0].set_ylim(0, 1)
    
    # Plot 2: Prediction distribution by group (stacked bar)
    dropout_rates = class_dist['Dropout_rate'].values
    enrolled_rates = class_dist['Enrolled_rate'].values
    graduate_rates = class_dist['Graduate_rate'].values
    
    axes[idx, 1].bar(x_pos, dropout_rates, label='Dropout', color='#d62728', alpha=0.8)
    axes[idx, 1].bar(x_pos, enrolled_rates, bottom=dropout_rates, 
                     label='Enrolled', color='#ff7f0e', alpha=0.8)
    axes[idx, 1].bar(x_pos, graduate_rates, 
                     bottom=dropout_rates + enrolled_rates,
                     label='Graduate', color='#2ca02c', alpha=0.8)
    axes[idx, 1].set_xticks(x_pos)
    axes[idx, 1].set_xticklabels(groups, rotation=45, ha='right')
    axes[idx, 1].set_ylabel('Prediction Rate')
    axes[idx, 1].set_title(f'Prediction Distribution by {attr_name}')
    axes[idx, 1].legend()
    axes[idx, 1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

print("  Left: Performance metrics (accuracy, macro F1) by group")
print("  Right: Prediction distribution (% predicted as each class) by group")

## 9.4 Fairness Summary and Recommendations

In [ ]:
print_section_header("Fairness Analysis Summary")

# Create summary DataFrame
fairness_summary = pd.DataFrame({
    'Protected Attribute': list(fairness_results.keys()),
    'Accuracy Range': [f"{results['by_group']['accuracy'].min():.3f} - {results['by_group']['accuracy'].max():.3f}" 
                       for results in fairness_results.values()],
    'Accuracy Disparity': [results['by_group']['accuracy'].max() - results['by_group']['accuracy'].min() 
                           for results in fairness_results.values()],
    'Macro F1 Range': [f"{results['by_group']['macro_f1'].min():.3f} - {results['by_group']['macro_f1'].max():.3f}"
                       for results in fairness_results.values()],
    'F1 Disparity': [results['by_group']['macro_f1'].max() - results['by_group']['macro_f1'].min()
                     for results in fairness_results.values()]
})

print("\n" + "="*80)
print("FAIRNESS SUMMARY")
print("="*80)
print(fairness_summary.to_string(index=False))
print("="*80)

# Identify potential fairness issues
print("\n" + "="*80)
print("POTENTIAL FAIRNESS CONCERNS")
print("="*80)

threshold_accuracy = 0.05  # Accuracy disparity threshold
threshold_f1 = 0.05  # F1 disparity threshold

concerns = []
for _, row in fairness_summary.iterrows():
    if row['Accuracy Disparity'] > threshold_accuracy:
        concerns.append(f"• {row['Protected Attribute']}: Significant accuracy disparity ({row['Accuracy Disparity']:.3f})")
    if row['F1 Disparity'] > threshold_f1:
        concerns.append(f"• {row['Protected Attribute']}: Significant F1 score disparity ({row['F1 Disparity']:.3f})")

if concerns:
    for concern in concerns:
        print(concern)
    print("\nRecommendation: Consider bias mitigation techniques or further investigation")
else:
    print("  All protected attributes show reasonable parity in predictions")

print("="*80)

# Final summary
print("\n" + "="*80)
print("FAIRNESS ANALYSIS COMPLETE")
print("="*80)
print(f"\nAnalyzed {len(protected_attrs)} protected attributes:")
print(', '.join(protected_attrs.keys()))
print("\nKey Metrics:")
print("- Accuracy Disparity: Difference in accuracy across groups")
print("- Macro F1 Disparity: Difference in F1 scores across groups")
print("- Prediction Distribution: Proportion predicted as each class per group")
print("\nInterpretation:")
print("- Disparity < 0.05: Generally considered fair")
print("- Disparity 0.05-0.10: Moderate concern, monitor closely")
print("- Disparity > 0.10: Significant concern, investigate further")
print("\nNext Steps if concerns identified:")
print("1. Collect more balanced training data")
print("2. Apply fairness constraints during training")
print("3. Post-process predictions to improve fairness")
print("4. Investigate root causes of disparities")
print("="*80)

### Project Summary and Key Findings

After comprehensive analysis and modeling, here are the key findings from this student dropout prediction project:

#### Model Selection

**LightGBM (Class-Weighted)** emerges as the best choice:
- **Overall Performance**: Weighted F1-score of 0.8148
- **Balanced Predictions**: Performs well across all three classes (Dropout, Enrolled, Graduate)
- **Efficiency**: Fast training and prediction times
- **Interpretability**: Provides clear feature importance rankings

#### Most Important Features

The analysis identified the following key predictors of student outcomes:

1. **Academic Performance**: Curricular units approved and grades (1st and 2nd semester)
2. **Enrollment Patterns**: Number of units enrolled and evaluations completed
3. **Engineered Features**: GPA, success rates, and course load metrics
4. **Tuition Status**: Whether tuition fees are up to date
5. **Admission Grade**: Initial academic standing

These features collectively explain the majority of variance in student outcomes.

#### Class Imbalance and SMOTE

- **Initial Challenge**: Strong class imbalance in the dataset (favoring "Graduate")
- **SMOTE Impact**: Improved minority class recall but decreased overall precision
- **Final Approach**: Class weighting without SMOTE provided the best balance

#### Fairness Analysis

**Gender Fairness**:
- Demographic parity difference: 0.0012 (excellent)
- Equal opportunity difference: 0.0036 (excellent)
- Model shows minimal bias across gender

**Age Fairness**:
- Moderate disparities detected between younger and older students
- Prediction rates differ by ~8.7% between age groups
- Consider age-specific interventions in practice

#### Key Learnings

1. **Feature Engineering**: Aggregated metrics (GPA, success rates) proved highly predictive
2. **Ensemble Methods**: Tree-based models (LightGBM, Random Forest) outperformed linear models
3. **Class Balancing**: Careful tuning of class weights outperformed synthetic sampling
4. **Interpretability**: Feature importance and permutation analysis provided actionable insights
5. **Fairness Matters**: Evaluating model fairness across demographics is essential for ethical ML

#### Potential Applications

This model could inform:
- Early warning systems for at-risk students
- Resource allocation for academic support services
- Personalized intervention strategies
- Understanding factors that contribute to student success

#### Limitations and Future Work

- Model performance varies by class (Enrolled is hardest to predict)
- Age-related fairness could be improved
- Temporal dynamics (how predictions change over time) not fully explored
- External validation on other institutions would strengthen generalizability
